In [1]:
import pandas as pd
import numpy as np
import talib
# import tqdm
from tqdm.notebook import tqdm

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15.0, 8.0) # 单位是inches

# 导入数据

In [2]:
data = pd.read_csv("../../数据获取/聚宽数据/最新数据.csv")
data=data.set_index(['Unnamed: 0', 'Unnamed: 1'])
print('共导入{}行数据,股票个数：{},平均每个股票的的数据量:{}'.format(
    len(data),
    len(data.index.levels[0]),
    len(data)//len(data.index.levels[0])
))
data.head()

共导入3704160行数据,股票个数：4136,平均每个股票的的数据量:895


date   open   high    low  close
Unnamed: 0  Unnamed: 1                                        
000001.XSHE 0           2018-04-18  10.89  11.04  10.72  10.93
            1           2018-04-19  10.95  11.11  10.86  10.90
            2           2018-04-20  10.94  11.01  10.65  10.79
            3           2018-04-23  10.74  11.04  10.70  11.00
            4           2018-04-24  11.06  11.35  11.01  11.28

# 算法

In [3]:
# 我这里只是统计第一个
data_one = data.loc[data.index.levels[0][0]].copy() # 这个是每一个股票的数据

# 这里首先添加均线，
for ma_day in [5,10,20,30,60,120]:
    # 插入多条均线
    data_one.insert(
        loc=len(data_one.columns),
        column='ma{}'.format(ma_day),
        value=talib.MA(data_one['close'], timeperiod=ma_day)
    )
# 然后这里是比较了。
data_one.insert(
    loc=len(data_one.columns),
    column='均线多头',
    value=(data_one['ma5'] > data_one['ma10'] ) & (data_one['ma10'] > data_one['ma20'] )& (data_one['ma20'] > data_one['ma30']) 
)
# 然后是遍历了
data_one[data_one['均线多头']]

,date,open,high,low,close,ma5,ma10,ma20,ma30,ma60,ma120,均线多头
Unnamed: 1,,,,,,,,,,,,
90,2018-08-27,9.67,10.07,9.62,10.07,9.332,8.949,8.8740,8.855000,8.922167,NaN,True
91,2018-08-28,9.98,10.07,9.88,9.93,9.536,9.073,8.9160,8.905333,8.925000,NaN,True
92,2018-08-29,9.89,9.97,9.80,9.91,9.724,9.216,8.9700,8.955667,8.927667,NaN,True
93,2018-08-30,9.91,9.99,9.73,9.77,9.872,9.346,9.0270,9.000333,8.929833,NaN,True
94,2018-08-31,9.78,9.96,9.75,9.78,9.892,9.474,9.0860,9.033333,8.928500,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...
890,2021-12-14,18.16,18.17,17.52,17.58,18.280,18.084,17.9490,17.938000,18.505667,19.294167,True
967,2022-04-12,16.05,16.26,15.66,15.92,16.208,15.691,15.1745,15.140667,15.887500,16.988417,True
968,2022-04-13,15.89,16.08,15.72,15.80,16.090,15.786,15.2400,15.142333,15.864333,16.956250,True


##  先用一个简单的数据

In [4]:

x = [True, False, True, True, True, False, False, True, True, True, True, True] # 一共12个数据
x = np.asanyarray(x)
x

array([ True, False,  True,  True,  True, False, False,  True,  True,
        True,  True,  True])

In [5]:
if x.ndim != 1:
    raise ValueError('only 1D array supported')

In [6]:
n = x.shape[0]
n

12

In [12]:
loc_run_start = np.empty(n, dtype=bool)
loc_run_start[0] = True 
loc_run_start # 这个上边构造的是随机的，真正赋值的只有第一项

array([ True,  True,  True,  True,  True,  True, False, False, False,
       False, False, False])

In [13]:
x[:-1] # 这个是截至到最后一项

array([ True, False,  True,  True,  True, False, False,  True,  True,
        True,  True])

In [15]:
x[1:] # 这个是从第2项目开始

array([False,  True,  True,  True, False, False,  True,  True,  True,
        True,  True])

In [16]:
np.not_equal(x[:-1], x[1:], out=loc_run_start[1:]) # 这个是将这游戏昂跟下一项比较，如果不同是True，结果放在loc_run_start

array([ True,  True, False, False,  True, False,  True, False, False,
       False, False])

In [17]:
loc_run_start # 这个比上边的结果多了一项，是因为loc_run_start[1:]，这个是第二项开始。

array([ True,  True,  True, False, False,  True, False,  True, False,
       False, False, False])

In [18]:
np.nonzero(loc_run_start) # 不是0的。

(array([0, 1, 2, 5, 7], dtype=int64),)

In [20]:
run_starts = np.nonzero(loc_run_start)[0]
run_starts

array([0, 1, 2, 5, 7], dtype=int64)

In [21]:
run_values = x[loc_run_start]
run_values

array([ True, False,  True, False,  True])

In [22]:
np.append(run_starts, n) # 只是添加了一个末尾

array([ 0,  1,  2,  5,  7, 12], dtype=int64)

In [23]:
np.diff(np.append(run_starts, n)) # 这个是比较相差多大的

array([1, 1, 3, 2, 5], dtype=int64)

In [24]:
run_lengths = np.diff(np.append(run_starts, n))
run_lengths

array([1, 1, 3, 2, 5], dtype=int64)

In [25]:
def find_runs(x):
    """找到连续的"""
    x = np.asanyarray(x) # 首先转成数组
    if x.ndim != 1:
        # 这里只是支持1维的数组
        raise ValueError('only 1D array supported')
    n = x.shape[0] # 维度，这里可以理解成这个一维的长度
    # 
    loc_run_start = np.empty(n, dtype=bool)
    loc_run_start[0] = True 
    loc_run_start # 这个上边构造的是随机的，真正赋值的只有第一项
    # 比较
    np.not_equal(x[:-1], x[1:], out=loc_run_start[1:]) # 这个是将这游戏昂跟下一项比较，如果不同是True，结果放在loc_run_start
    # 相同的是False，不同的是True，这里取得的是不同的位置
    run_starts = np.nonzero(loc_run_start)[0]
    # 这个是值
    run_values = x[loc_run_start]
    # 长度
    run_lengths = np.diff(np.append(run_starts, n))
    return run_starts, run_values, run_lengths
    

In [26]:
find_runs(x) 

(array([0, 1, 2, 5, 7], dtype=int64),
 array([ True, False,  True, False,  True]),
 array([1, 1, 3, 2, 5], dtype=int64))

In [27]:
find_runs(data_one['均线多头']) # 这个是最快速的方法，

(array([  0,  90,  98, 111, 117, 127, 132, 190, 218, 223, 225, 233, 250,
        287, 299, 304, 306, 310, 317, 322, 330, 345, 347, 354, 367, 372,
        374, 378, 380, 414, 415, 416, 426, 493, 503, 538, 545, 579, 597,
        605, 616, 619, 624, 630, 643, 668, 690, 735, 746, 750, 760, 764,
        767, 849, 851, 857, 859, 889, 891, 967, 971], dtype=int64),
 array([False,  True, False,  True, False,  True, False,  True, False,
         True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False,  True, False,  True, False,  True, False,
         True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False,  True, False,  True, False,  True, False,
         True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False,  True, False,  True, False]),
 array([90,  8, 13,  6, 10,  5, 58, 28,  5,  2,  8, 17, 37, 12,  5,  2,  4,
         7,  5,  8, 15,  2,  7, 13,  5,  2,  4,  2, 34,  1,  1, 10, 67, 10,
   

## 算法的实质
np.not_equal 比较的是前后不相同的数据，也就是两组不同数据之间的分割线。